In [8]:
pip install Cython numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 4.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pystan==2.19.1.1

  Using cached pystan-2.19.1.1.tar.gz (16.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      Cython>=0.22 and NumPy are required.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

# Load dataset
housing_data = pd.read_csv('/Users/mariecieslar/Desktop/Schreibtisch - Maries MacBook Air/Bocconi/Semester 1/STATS/Final Project/housing.csv')

# Encode categorical variables
housing_data['Type'] = housing_data['Type'].astype('category').cat.codes
housing_data['Regionname'] = housing_data['Regionname'].astype('category').cat.codes

# Prepare data for Stan
stan_data = {
    'N': len(housing_data),
    'K': housing_data['Regionname'].nunique(),
    'region': housing_data['Regionname'] + 1,  # Stan uses 1-based indexing
    'rooms': housing_data['Rooms'].values,
    'type': housing_data['Type'].values,
    'y': housing_data['BuildingArea'].fillna(-1).values,  # Placeholder for missing data
}



In [2]:
import pystan
import matplotlib.pyplot as plt

# Compile the Stan model
sm = pystan.StanModel(model_code=stan_code)

# Fit the model
fit = sm.sampling(data=stan_data, iter=2000, chains=4, warmup=1000, thin=1, seed=42)

# Print the summary of results
print(fit)

# Extract posterior samples
samples = fit.extract()

# Plot region-level effects
plt.hist(samples['region_effect'], bins=30, density=True)
plt.title('Posterior Distribution of Region Effects')
plt.xlabel('Region Effect')
plt.ylabel('Density')
plt.show()

ModuleNotFoundError: No module named 'pystan'

In [ ]:
# Step 2: Preprocess the data
# Encode categorical variables
label_encoder_type = LabelEncoder()
label_encoder_region = LabelEncoder()
housing_data['Type'] = label_encoder_type.fit_transform(housing_data['Type'])
housing_data['Regionname'] = label_encoder_region.fit_transform(housing_data['Regionname'])

# Extract relevant variables
rooms = housing_data['Rooms'].values
type_encoded = housing_data['Type'].values
region = housing_data['Regionname'].values
building_area = housing_data['BuildingArea'].values

# Create a mask for observed and missing data
observed_mask = ~np.isnan(building_area)


In [23]:
housing_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [ ]:
# Step 3: Define the hierarchical Bayesian model
with pm.Model() as hierarchical_model:
    # Global priors
    intercept = pm.Normal('intercept', mu=100, sigma=50)  # Prior for global intercept
    beta_rooms = pm.Normal('beta_rooms', mu=0, sigma=10)  # Prior for the Rooms effect
    beta_type = pm.Normal('beta_type', mu=0, sigma=10)  # Prior for the Type effect

    # Group-level effects for Regionname
    region_mu = pm.Normal('region_mu', mu=0, sigma=20, shape=len(np.unique(region)))  # Group means
    region_sigma = pm.HalfNormal('region_sigma', sigma=10)  # Shared standard deviation
    region_effect = pm.Normal(
        'region_effect', mu=region_mu[region], sigma=region_sigma, shape=len(region)
    )  # Group-level effect for each region
    
    # Model equation for BuildingArea
    mu = (
        intercept
        + beta_rooms * rooms
        + beta_type * type_encoded
        + region_effect
    )  # Linear model with group-level effects
    
    # Likelihood for observed BuildingArea
    sigma = pm.HalfNormal('sigma', sigma=30)  # Prior for residual standard deviation
    observed_building_area = pm.Normal(
        'observed_building_area', mu=mu[observed_mask], sigma=sigma, observed=building_area[observed_mask]
    )  # Likelihood for observed data
    
    # Imputation for missing values
    missing_building_area = pm.Normal(
        'missing_building_area', mu=mu[~observed_mask], sigma=sigma, shape=(~observed_mask).sum()
    )  # Missing values treated as latent variables

    # Step 4: Posterior sampling
    trace = pm.sample(2000, tune=1000, return_inferencedata=True)


In [ ]:
# Step 5: Impute missing values with posterior means
imputed_building_area = trace.posterior['missing_building_area'].mean(dim=['chain', 'draw']).values
housing_data.loc[np.isnan(housing_data['BuildingArea']), 'BuildingArea'] = imputed_building_area